In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from tensorflow.keras.applications import MobileNetV3Small
import os
import seaborn as sns
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [ ]:
print(os.listdir("dataset/split/"))
SIZE = 224

In [ ]:
train_images = []
train_labels = []

for directory_path in glob.glob("dataset/split/train/*"):
    label = directory_path.split("\\")[-1]
    # print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        # print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob("dataset/split/test/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

In [ ]:
for layer in model.layers:
	layer.trainable = False

model.summary()

In [ ]:
feature_extractor=model.predict(x_train)
features = feature_extractor.reshape(feature_extractor.shape[0], -1)
X_for_RF = features

In [ ]:
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
RF_model.fit(X_for_RF, y_train)

In [ ]:
X_test_feature = model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
prediction_RF = RF_model.predict(X_test_features)
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print ("Accuracy = ", metrics.accuracy_score(test_labels, prediction_RF))